In [26]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack


In [27]:

class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train = train.head(2000)
test = test.head(2000)

Used the test data to train which shouln't happen

In [28]:
train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])


In [29]:

word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    # token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000)

word_vectorizer.fit(all_text)

train_features = word_vectorizer.transform(train_text)
test_features = word_vectorizer.transform(test_text)



Output isn't right too, hasn't been rounded to 1 or 0 and is in the wrong state it seems as all numbers of one column, ex. Threat, are simlar to each other. As well it gets all columns not just Obcense and Threat.  

In [30]:

scores = []
out = pd.DataFrame.from_dict({'id': test['id']})
for class_name in class_names:
    train_target = train[class_name]
    classifier = LogisticRegression(C=0.1, solver='sag')

    cv_score = np.mean(cross_val_score(classifier, train_features, train_target, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(train_features, train_target)
    out[class_name] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))

out.to_csv('out.csv', index=False)

CV score for class toxic is 0.912376505930657
CV score for class severe_toxic is 0.9545574795574795
CV score for class obscene is 0.9524278187506511
CV score for class threat is 0.9648165310815914
CV score for class insult is 0.9466200404212318
CV score for class identity_hate is 0.8836125727248642
Total CV score is 0.9357351580777458
